In [1]:
print("OK!")

OK!


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


d:\Softwares\anaconda3\envs\mchatbot\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
PINECONE_API_KEY = "2bd5fd9f-2c56-42f9-aa1b-037960262fff"
PINECONE_API_ENV = "gcp-starter"

In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf("data/")

In [7]:
# extracted_data

In [8]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [11]:
# text_chunks

In [12]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

In [14]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [15]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [17]:
# query_result

In [18]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="medical-chatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [19]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={}), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drug

In [20]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [21]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [22]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [23]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [24]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Response :  Acne is a common skin disease characterized by pimples on the face, chest, and back that occur when the pores of the skin become clogged with oil, dead skin cells, and bacteria.


KeyboardInterrupt: 

In [1]:
from langchain.vectorstores import chroma
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader


In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
text_spiltter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 200)

In [5]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf("data/")

In [7]:
text = text_spiltter.split_documents(extracted_data)

In [8]:
text[1].page_content

'Survey of Pharma Clusters  \nCentre for Mar ket Res earch & Social Development                                                                                 \n ACKNOWLEDGEMENTS  \n \n \nCentre for Market Research & Social Development, New Delhi, wishes to express its \nsincere gratitude to Department of Pharmaceuticals , Governm ent of India, for entrusting \nthis work to us.  \n \nWe are greatly indebted to all the respondents of the study for their kind cooperation and \nsupport extended towards this assignment. We are also thankful to state officials  for \nproviding us with the best supp ort data and information required for the study.  \n \nWe gratefully acknowledge the professional contributions of all the staff of Centre for \nMarket Research & Social Development for their involvement in successful completion of \nthe study.   \n \nFebruary 2023   \nDr. Girija Bhusan Nanda  \n(Director)'

In [9]:
len(text)

119

In [10]:
from langchain.vectorstores import Chroma

In [11]:
from langchain import embeddings

In [74]:
class download_hugging_face_embeddings():
  def __call__(self,input):
    embeddings = HuggingFaceEmbeddings(model_name=input)
    return embeddings

In [75]:
embedding = download_hugging_face_embeddings()

In [76]:
embedding_func = embedding("sentence-transformers/all-MiniLM-L6-v2")


c:\users\administrator\downloads\end-to-end-medical-product\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
import chromadb

In [17]:
from chromadb import Client

In [4]:
client = Client()

In [26]:
client = chromadb.PersistentClient(path="data/DB")

In [15]:
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

In [36]:
embedding_function = SentenceTransformerEmbeddingFunction()

In [17]:
chroma_client = chromadb.Client()

In [24]:
chroma_collection = chroma_client.create_collection(name="db_data", embedding_function=embedding_function)

In [18]:
ids = [str(i) for i in range(len(text))]

In [19]:
document = [text[i].page_content for i in range(len(text))]

In [1]:
documents = ["This is a sample document.", "Another document for testing."]

In [80]:
ids = [str(i) for i in range(len(documents))]

NameError: name 'documents' is not defined

In [28]:
from langchain.embeddings import OpenAIEmbeddings

In [31]:
import os
os.environ['OPENAI_API_KEY']="sk-WTEyTlDKpt0UBmY_2E7yFgpU7iuplEdu6Z3N1xBz6bT3BlbkFJ_WZ_Um7sQFtnj6d90UDF0EW9O8nQlh69XAukp7kIcA"

In [39]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [40]:
embedding_function = OpenAIEmbeddingFunction(
    api_key=os.environ.get('OPENAI_API_KEY'),
    model_name="text-embedding-ada-002"
)

In [93]:
chroma_collection=chroma_collection.add( ids=ids, documents=document)

Insert of existing embedding ID: 0
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Insert of existing embedding ID: 4
Insert of existing embedding ID: 5
Insert of existing embedding ID: 6
Insert of existing embedding ID: 7
Insert of existing embedding ID: 8
Insert of existing embedding ID: 9
Insert of existing embedding ID: 10
Insert of existing embedding ID: 11
Insert of existing embedding ID: 12
Insert of existing embedding ID: 13
Insert of existing embedding ID: 14
Insert of existing embedding ID: 15
Insert of existing embedding ID: 16
Insert of existing embedding ID: 17
Insert of existing embedding ID: 18
Insert of existing embedding ID: 19
Insert of existing embedding ID: 20
Insert of existing embedding ID: 21
Insert of existing embedding ID: 22
Insert of existing embedding ID: 23
Insert of existing embedding ID: 24
Insert of existing embedding ID: 25
Insert of existing embedding ID: 26
Insert of existing embedding ID: 27
In

In [20]:
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA

In [94]:
retriever = chroma_collection.as_retriever()
docs = retriever.get_relevant_documents("who is the presisdent")

AttributeError: 'NoneType' object has no attribute 'as_retriever'

In [59]:
import shutil
import os

# Optionally clear the directory before re-initializing
if os.path.exists(persist_directory):
    shutil.rmtree(persist_directory)

In [28]:
chromadb.EphemeralClient()=None

SyntaxError: cannot assign to function call (376350678.py, line 1)

In [29]:
self._client = chromadb.Client(self._client_settings)
client = chromadb.EphemeralClient(settings=setts)

In [78]:
persist_directory = 'db'

In [79]:

vectordb = None

vectordb = Chroma.from_documents(documents=text, embedding = embedding_func, persist_directory = persist_directory)

#vectordb.persist()
#vectordb = none
#vectordb = Chroma(persist_directory = persist_directory, embedding_function = embeddings)



ValueError: An instance of Chroma already exists for ephemeral with different settings

In [60]:
vectordb = Chroma(persist_directory = persist_directory, embedding_function = embeddings)

ValueError: An instance of Chroma already exists for ephemeral with different settings

In [49]:
collection_names = chroma_client.list_collections()

In [50]:
print("Existing collections:", collection_names)

Existing collections: [Collection(name=emedded_db)]


In [51]:
chroma_client.delete_collection(name="emedded_db")

In [23]:
chroma_client.count_collections()

0

In [25]:
chromadb.EphemeralClient()